# Modelling

In [ ]:
# main source: Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
# secondary source: https://gist.github.com/aniruddha27/eaf96b6ce2a98eb8cded822d01493a70#file-auc-roc6-py

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
%matplotlib inline

In [2]:
#load in data

df_rp = pd.read_csv("Desktop/Feature extraction/df_regionprops.csv")
df_surv_2 = pd.read_csv('Desktop/Feature extraction/with less features/df_surv_2.csv')
df_surv_3 = pd.read_csv('Desktop/Feature extraction/with less features/df_surv_3.csv')
df_all = pd.read_csv('Desktop/Feature extraction/with less features/df_all.csv')
df_all = df_all.drop(["Unnamed: 0"], axis = 1)
df_all['Survival_group'] = df_surv_2.iloc[:,5] #switch between 2-class survival and 3-class survival

# Creating 3 feature sets

In [3]:
# 1. Creating full radiomics feature set

df_radiomics= df_all.copy()



In [4]:
# 2. Creating shape and volume feature set

df_2 = pd.concat([df_all, df_rp], axis = 1, join = "inner")

df_2.drop(df_2.columns[df_2.columns.str.contains('original_glcm')], axis=1, inplace=True)
df_sha_vol = df_2.drop(['centroid_0_ncrnet', 'centroid_0_ed', 'centroid_0_et',
       'centroid_1_ncrnet', 'centroid_1_ed', 'centroid_1_et',
       'centroid_2_ncrnet', 'centroid_2_ed', 'centroid_2_et', 'area_ncrnet',
       'area_ed', 'area_et', 'major_axis_length_ncrnet',
       'major_axis_length_ed', 'major_axis_length_et',
       'minor_axis_length_ncrnet', 'minor_axis_length_ed',
       'minor_axis_length_et', 'Unnamed: 0','original_firstorder_10Percentile_ed', 
       'original_firstorder_90Percentile_ed', 'original_firstorder_Energy_ed', 'original_firstorder_Entropy_ed',
       'original_firstorder_InterquartileRange_ed', 'original_firstorder_Kurtosis_ed',
       'original_firstorder_Maximum_ed', 'original_firstorder_MeanAbsoluteDeviation_ed',
       'original_firstorder_Mean_ed', 'original_firstorder_Median_ed', 'original_firstorder_Minimum_ed',
       'original_firstorder_Range_ed', 'original_firstorder_RobustMeanAbsoluteDeviation_ed',
       'original_firstorder_RootMeanSquared_ed', 'original_firstorder_Skewness_ed',
       'original_firstorder_TotalEnergy_ed', 'original_firstorder_Uniformity_ed', 
       'original_firstorder_Variance_ed','original_firstorder_10Percentile_ncrnet',
       'original_firstorder_90Percentile_ncrnet', 'original_firstorder_Energy_ncrnet', 'original_firstorder_Entropy_ncrnet',
       'original_firstorder_InterquartileRange_ncrnet', 'original_firstorder_Kurtosis_ncrnet', 'original_firstorder_Maximum_ncrnet',
       'original_firstorder_MeanAbsoluteDeviation_ncrnet', 'original_firstorder_Mean_ncrnet', 'original_firstorder_Median_ncrnet',
       'original_firstorder_Minimum_ncrnet', 'original_firstorder_Range_ncrnet', 'original_firstorder_RobustMeanAbsoluteDeviation_ncrnet',
       'original_firstorder_RootMeanSquared_ncrnet', 'original_firstorder_Skewness_ncrnet', 'original_firstorder_TotalEnergy_ncrnet', 
       'original_firstorder_Uniformity_ncrnet', 'original_firstorder_Variance_ncrnet',
       'original_firstorder_10Percentile_et', 
       'original_firstorder_90Percentile_et', 'original_firstorder_Energy_et', 'original_firstorder_Entropy_et',
       'original_firstorder_InterquartileRange_et', 'original_firstorder_Kurtosis_et',
       'original_firstorder_Maximum_et', 'original_firstorder_MeanAbsoluteDeviation_et',
       'original_firstorder_Mean_et', 'original_firstorder_Median_et', 'original_firstorder_Minimum_et',
       'original_firstorder_Range_et', 'original_firstorder_RobustMeanAbsoluteDeviation_et',
       'original_firstorder_RootMeanSquared_et', 'original_firstorder_Skewness_et',
       'original_firstorder_TotalEnergy_et', 'original_firstorder_Uniformity_et', 
       'original_firstorder_Variance_et'], axis=1)


In [5]:
# 3. Creating handcrafted model (regionprops)

df_handcrafted = df_2.copy()
df_handcrafted.drop(df_handcrafted.columns[df_handcrafted.columns.str.contains('original_')], axis=1, inplace=True)
df_handcrafted = df_handcrafted.drop(['Unnamed: 0'], axis=1)


In [6]:
#create 'NA' group in 'Extent_of_Resection' column and use it as group

df_radiomics.Extent_of_Resection = df_radiomics.Extent_of_Resection.fillna("NA")
df_sha_vol.Extent_of_Resection = df_sha_vol.Extent_of_Resection.fillna("NA")
df_handcrafted.Extent_of_Resection = df_handcrafted.Extent_of_Resection.fillna("NA")

# Train/test split, normalization and encoding

In [7]:
# Encoding Extent_of_Resection

df_radiomics_enc = pd.get_dummies(df_radiomics['Extent_of_Resection'])
df_radiomics = pd.concat([df_radiomics, df_radiomics_enc], axis = 1).drop(['Extent_of_Resection'], axis=1)

df_sha_vol_enc = pd.get_dummies(df_sha_vol['Extent_of_Resection'])
df_sha_vol = pd.concat([df_sha_vol, df_sha_vol_enc], axis = 1).drop(['Extent_of_Resection'], axis=1)

df_handcrafted_enc = pd.get_dummies(df_handcrafted['Extent_of_Resection'])
df_handcrafted = pd.concat([df_handcrafted, df_handcrafted_enc], axis = 1).drop(['Extent_of_Resection'], axis=1)

#re-arrange order of columns

cols = df_radiomics.columns.tolist()
cols = cols[-3:] + cols[:-3]
df_radiomics = df_radiomics[cols]

cols = df_sha_vol.columns.tolist()
cols = cols[-3:] + cols[:-3]
df_sha_vol = df_sha_vol[cols]

cols = df_handcrafted.columns.tolist()
cols = cols[-3:] + cols[:-3]
df_handcrafted = df_handcrafted[cols]



# RFE-reduced feature sets

In [ ]:
# 2-class RFE-reduced radiomics set --> 23 features

# df_radiomics_rfe2 = df_radiomics.copy()
# df_radiomics_rfe2 = df_radiomics_rfe2[['NA', 'STR', 'Age', 'original_shape_Elongation_ncrnet',
#        'original_shape_Maximum2DDiameterRow_ncrnet',
#        'original_shape_Maximum3DDiameter_ncrnet',
#        'original_shape_SurfaceArea_ncrnet',
#        'original_shape_VoxelVolume_ncrnet',
#        'original_firstorder_Maximum_ncrnet', 'original_glcm_Contrast_ncrnet',
#        'original_glcm_DifferenceAverage_ncrnet',
#        'original_shape_MajorAxisLength_ed',
#        'original_shape_Maximum2DDiameterColumn_ed',
#        'original_shape_MinorAxisLength_ed', 'original_shape_SurfaceArea_ed',
#        'original_shape_SurfaceVolumeRatio_ed',
#        'original_firstorder_Maximum_ed', 'original_firstorder_Mean_ed',
#        'original_firstorder_Variance_ed', 'original_shape_LeastAxisLength_et',
#        'original_shape_MajorAxisLength_et',
#        'original_shape_Maximum2DDiameterRow_et',
#        'original_shape_Maximum2DDiameterSlice_et',
#        'original_shape_MeshVolume_et', 'original_shape_SurfaceArea_et',
#        'original_shape_SurfaceVolumeRatio_et',
#        'original_shape_VoxelVolume_et']]

In [ ]:
# 2-class RFE-reduced shape & volume set --> 38 features 

# df_sha_vol_rfe2 = df_sha_vol.copy()
# df_sha_vol_rfe2 = df_sha_vol_rfe2[['GTR', 'NA', 'Age', 'original_shape_Elongation_ncrnet',
# 'original_shape_Flatness_ncrnet','original_shape_LeastAxisLength_ncrnet',
#       'original_shape_Maximum2DDiameterColumn_ncrnet',
#        'original_shape_Maximum2DDiameterRow_ncrnet',
#        'original_shape_Maximum2DDiameterSlice_ncrnet',
#        'original_shape_Maximum3DDiameter_ncrnet',
#        'original_shape_SurfaceArea_ncrnet',
#        'original_shape_SurfaceVolumeRatio_ncrnet',
#        'original_shape_VoxelVolume_ncrnet', 'original_shape_Elongation_ed',
#        'original_shape_Flatness_ed', 'original_shape_LeastAxisLength_ed',
#        'original_shape_MajorAxisLength_ed',
#        'original_shape_Maximum2DDiameterColumn_ed',
#        'original_shape_Maximum2DDiameterRow_ed',
#        'original_shape_Maximum2DDiameterSlice_ed',
#        'original_shape_Maximum3DDiameter_ed', 'original_shape_MeshVolume_ed',
#        'original_shape_MinorAxisLength_ed', 'original_shape_SurfaceArea_ed',
#        'original_shape_SurfaceVolumeRatio_ed', 'original_shape_VoxelVolume_ed',
#        'original_shape_Elongation_et', 'original_shape_LeastAxisLength_et',
#        'original_shape_MajorAxisLength_et',
#        'original_shape_Maximum2DDiameterColumn_et',
#        'original_shape_Maximum2DDiameterRow_et',
#        'original_shape_Maximum2DDiameterSlice_et',
#        'original_shape_Maximum3DDiameter_et', 'original_shape_MeshVolume_et',
#        'original_shape_MinorAxisLength_et', 'original_shape_SurfaceArea_et',
#        'original_shape_SurfaceVolumeRatio_et',
#        'original_shape_VoxelVolume_et']]

In [ ]:
# 2-class RFE-reduced handcrafted set --> 9 features 

# df_handcrafted_rfe2 = df_handcrafted.copy()
# df_handcrafted_rfe2 = df_handcrafted_rfe2[['STR', 'Age', 'centroid_0_ncrnet', 'centroid_0_et', 'centroid_2_ed',
#        'centroid_2_et', 'area_ed', 'major_axis_length_ncrnet',
#        'minor_axis_length_ed']]

In [ ]:
#3-class RFE-reduced radiomics set --> 3 features

# df_radiomics_rfe = df_radiomics.copy()
# df_radiomics_rfe = df_radiomics_rfe[["Age", "STR", "original_shape_LeastAxisLength_ed"]]

In [ ]:
# #3-class RFE-reduced shape and volume set --> 8 features 

# df_sha_vol_rfe = df_sha_vol.copy()
# df_sha_vol_rfe = df_sha_vol_rfe[['STR', 'Age', 'original_shape_Maximum3DDiameter_ncrnet',
#        'original_shape_LeastAxisLength_ed',
#        'original_shape_Maximum2DDiameterSlice_et',
#        'original_shape_MeshVolume_et', 'original_shape_SurfaceArea_et',
#        'original_shape_SurfaceVolumeRatio_et']]

In [ ]:
#3-class RFE-reduced handcrafted set --> 4 features ('GTR', 'STR', 'Age', 'minor_axis_length_ed')

# df_handcrafted_rfe = df_handcrafted.copy()
# df_handcrafted_rfe = df_handcrafted_rfe[['GTR', 'STR', 'Age', 'minor_axis_length_ed']]

# ML Modelling

In [11]:
#Initialize X and y datasets

df_X = df_radiomics.copy()
df_X = df_X.drop(['Brats20ID', 'Survival_days', 'Survival_group'], axis = 1)

df_y = df_radiomics.copy()
df_y = df_y.iloc[:,6]

In [ ]:
#Train-test split

X = df_X
y = df_y


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#LabelEncode the target variable (3 classes) or (2 classes)
#0 = long-term   #0 = long-term
#1 = mid-term    #1 = short-term
#2 = short-term

def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

In [ ]:
#Optional: binarize the target variable for multiclass AUROC values

# y_train_bi = label_binarize(y_train, classes=['short-term','mid-term','long-term'])
# n_classes = y_train_bi.shape[1]

# y_test_bi = label_binarize(y_test, classes=['short-term', 'mid-term','long-term'])
# n_classes = y_test_bi.shape[1]

In [ ]:
# Initialize portions of dataset for StandardScaler

X_train_scaler = X_train.iloc[:,1:]
X_test_scaler = X_test.iloc[:,1:]

X_train_rest = X_train.iloc[:,:1]
X_test_rest = X_test.iloc[:,:1]

In [ ]:
#use StandardScaler and recombine non-scaled data with scaled data


scaler = StandardScaler()

X_train_st = pd.DataFrame(scaler.fit_transform(X_train_scaler),index=X_train_scaler.index)
X_test_st = pd.DataFrame(scaler.transform(X_test_scaler), index=X_test_scaler.index)

X_train_st = pd.DataFrame(data=X_train_st.values, columns=X_train_scaler.columns, index=X_train_scaler.index)
X_test_st = pd.DataFrame(data=X_test_st.values, columns=X_test_scaler.columns, index=X_test_scaler.index)


X_train_final = pd.concat([X_train_rest, X_train_st], axis = 1, join = "inner")
X_test_final = pd.concat([X_test_rest, X_test_st], axis = 1, join = "inner")


print(X_train_final.shape)
print(X_test_final.shape)

In [ ]:
#Plot distribution of labels in train and test data

plt.figure(figsize=(8,8))

plt.title("Training survival group distribution")
sns.countplot(x=y_train, order=('short-term','long-term'))
plt.savefig('Training survival group distribution 2-class.png', facecolor = 'w')
plt.show()

plt.figure(figsize=(8,8))
plt.title("Testing survival group distribution")
sns.countplot(x=y_test, order=('short-term','long-term'))
plt.savefig('Testing survival group distribution 2-class.png', facecolor = 'w')
plt.show()

# Feature selection

# 1. RFE-SVM

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV



svc = SVC(kernel="linear")
min_features_to_select=1
rfecv = RFECV(estimator=svc, step=1, cv=10,
              scoring='accuracy',
              min_features_to_select=1)

rfecv.fit(X_train_final, y_train_enc)

print("Optimal number of features : %d" % rfecv.n_features_)
print("Ranking of features:", rfecv.ranking_)

In [ ]:
# Plot number of features versus cross-validation scores
plt.figure()
plt.xlabel("Number of features")
plt.xlim(0,80)
plt.xticks(np.arange(0, 80, 5))
plt.ylabel("Cross validation accuracy")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
# plt.savefig('RFE_SVM plot.png', facecolor='w')
plt.show()

In [ ]:
print(rfecv.ranking_)

In [ ]:
print(rfecv.support_)

In [ ]:
print ('coefficients',rfecv.estimator_.coef_)

In [ ]:
print("Features to select:",X_train_final.columns[rfecv.support_])

# Independent feature importance

In [ ]:
#dataset with ALL features combined for feature importance

df_cor_all = df_radiomics.copy()
df_cor_all = pd.concat([df_cor_all, df_rp], axis=1, join='inner')
df_cor_all = df_cor_all.drop(['Brats20ID', 'Survival_group'], axis = 1)

In [ ]:
corr_matrix = df_cor_all.corr(method = 'spearman')
cor_all = corr_matrix["Survival_days"].sort_values(ascending=True)

print(cor_all)

In [ ]:
#Calculate Spearman's correlation to rank feature importances

from scipy.stats import spearmanr

r_cor = spearmanr(df_cor_all)
r_cor_list = list(r_cor)

display(r_cor_list)

In [ ]:
corr_matrix = df_radiomics.corr(method = 'spearman')
# print(corr_matrix["Survival_days"].sort_values(ascending=True))

corr_list = dict(corr_matrix["Survival_days"].sort_values(ascending=True))

print(corr_list)

# Modelling

# Baseline model

In [ ]:
#majority class classifier as baseline

from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="stratified", random_state=42)
dummy_clf.fit(X_train_final, y_train_enc)
dummy_clf.predict(X_test_final)
result = dummy_clf.score(X_test_final, y_test_enc)
print("The classification accuracy of a new data point =", result)

# Classification ROC_AUC

# k-NN ROC_AUC

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np

n_classes = y_train_bi.shape[1]

param_grid = {
    'estimator__n_neighbors': np.arange(1,144,1),
}

knn = OneVsRestClassifier(KNeighborsClassifier())

knn_clf = GridSearchCV(
        knn, param_grid, cv = 10, scoring='roc_auc')

knn_y_score = knn_clf.fit(X_train_final_rfe_rad, y_train_bi).predict_proba(X_test_final_rfe_rad)
y_true, y_pred = y_test_bi, knn_clf.predict(X_test_final_rfe_rad)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

knn_fpr = dict()
knn_tpr = dict()
knn_roc_auc = dict()
for i in range(n_classes):
    knn_fpr[i], knn_tpr[i], _ = roc_curve(y_test_bi[:, i], knn_y_score[:, i])
    knn_roc_auc[i] = auc(knn_fpr[i], knn_tpr[i])
    

knn_mean_roc = roc_auc_score(y_true, knn_clf.predict_proba(X_test_final_rfe_rad), multi_class='ovr')
knn_mean_roc_micro = roc_auc_score(y_true, knn_clf.predict_proba(X_test_final_rfe_rad), multi_class='ovr', average = 'micro')

print("Dictionary of ROC_AUC score per class:")
print(knn_roc_auc)
print("ROC_AUC score over all three labels [Macro-averaging]:")
print(knn_mean_roc)
print("ROC_AUC score over all three labels with unbalanced classes taken into account [Micro-averaging]: ")
print(knn_mean_roc_micro)

In [ ]:
# initializing macro averaging for plotting
import numpy as np

knn_all_fpr = np.unique(np.concatenate([knn_fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
knn_mean_tpr = np.zeros_like(knn_all_fpr)
for i in range(n_classes):
    knn_mean_tpr += np.interp(knn_all_fpr, knn_fpr[i], knn_tpr[i])

# Finally average it and compute AUC
knn_mean_tpr /= n_classes

knn_fpr["macro"] = knn_all_fpr
knn_tpr["macro"] = knn_mean_tpr
knn_roc_auc["macro"] = auc(knn_fpr["macro"], knn_tpr["macro"])

print(knn_roc_auc['macro'])

In [ ]:
# initializing micro averaging for plotting

knn_fpr["micro"], knn_tpr["micro"], _ = roc_curve(y_test_bi.ravel(), knn_y_score.ravel())
knn_roc_auc["micro"] = auc(knn_fpr["micro"], knn_tpr["micro"])

print(knn_roc_auc['micro'])

In [ ]:
#source: https://gist.github.com/aniruddha27/eaf96b6ce2a98eb8cded822d01493a70#file-auc-roc6-py

plt.figure(figsize=(8,8))
plt.plot(knn_fpr[0], knn_tpr[0], linestyle='-',color='black', label='Long-term ROC curve')
plt.plot(knn_fpr[1], knn_tpr[1], linestyle='-',color='black', label='Mid-term ROC curve')
plt.plot(knn_fpr[2], knn_tpr[2], linestyle='-',color='black', label='Short-term ROC curve')
plt.plot(knn_fpr['micro'], knn_tpr['micro'], linestyle='-',color='red', label='Micro avg ROC curve' )
plt.plot(knn_fpr['macro'], knn_tpr['macro'], linestyle = '-', color = 'brown', label='Macro avg ROC curve')
plt.title('ROC curve k-NN short-term survivor')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.plot(ident,ident, color='green', linewidth = 2)
# plt.savefig('ROC curve k-NN long-term survivor.png',dpi=300, facecolor = 'w')
plt.show()

# SVC ROC_AUC

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
import numpy as np


param_grid = {'estimator__kernel': ['linear','rbf','poly','sigmoid'],
              'estimator__gamma': [1, 0.1, 0.01,0.001],
              'estimator__C': [0.1, 1, 10, 100]}
    
svc = OneVsRestClassifier(SVC(kernel='linear', probability=True,
                                 random_state=42))


svc_clf = GridSearchCV(svc, param_grid, cv = 10, scoring='roc_auc')

svc_y_score = svc_clf.fit(X_train_final, y_train_bi).decision_function(X_test_final)
y_true, y_pred = y_test_bi, svc_clf.predict(X_test_final)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


svc_fpr = dict()
svc_tpr = dict()
svc_roc_auc = dict()
for i in range(n_classes):
    svc_fpr[i], svc_tpr[i], _ = roc_curve(y_test_bi[:, i], svc_y_score[:, i])
    svc_roc_auc[i] = auc(svc_fpr[i], svc_tpr[i])

svc_mean_roc = roc_auc_score(y_true, svc_clf.decision_function(X_test_final), multi_class='ovr')
svc_mean_roc_micro = roc_auc_score(y_true, svc_clf.decision_function(X_test_final), multi_class='ovr', average = 'micro')

print("Dictionary of ROC_AUC score per class:")
print(svc_roc_auc)
print("ROC_AUC score over all three labels [Macro-averaging]:")
print(svc_mean_roc)
print("ROC_AUC score over all three labels with unbalanced classes taken into account [Micro-averaging]: ")
print(svc_mean_roc_micro)

In [ ]:
# initializing macro averaging for plotting
import numpy as np

svc_all_fpr = np.unique(np.concatenate([svc_fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
svc_mean_tpr = np.zeros_like(svc_all_fpr)
for i in range(n_classes):
    svc_mean_tpr += np.interp(svc_all_fpr, svc_fpr[i], svc_tpr[i])

# Finally average it and compute AUC
svc_mean_tpr /= n_classes

svc_fpr["macro"] = svc_all_fpr
svc_tpr["macro"] = svc_mean_tpr
svc_roc_auc["macro"] = auc(svc_fpr["macro"], svc_tpr["macro"])

print(svc_roc_auc["macro"])

In [ ]:
# initializing micro averaging for plotting

svc_fpr["micro"], svc_tpr["micro"], _ = roc_curve(y_test_bi.ravel(), svc_y_score.ravel())
svc_roc_auc["micro"] = auc(svc_fpr["micro"], svc_tpr["micro"])

print(svc_roc_auc["micro"])

In [ ]:
#source: https://gist.github.com/aniruddha27/eaf96b6ce2a98eb8cded822d01493a70#file-auc-roc6-py

plt.figure(figsize=(8,8))

plt.plot(svc_fpr[0], svc_tpr[0], linestyle='-',color='black', label='Long-term ROC curve')
plt.plot(svc_fpr[1], svc_tpr[1], linestyle='-',color='black', label='Mid-term ROC curve')
plt.plot(svc_fpr[2], svc_tpr[2], linestyle='-',color='black', label='Short-term ROC curve')
plt.plot(svc_fpr['micro'], svc_tpr['micro'], linestyle='-',color='red', label='Micro avg ROC curve' )
plt.plot(svc_fpr['macro'], svc_tpr['macro'], linestyle = '-', color = 'brown', label='Macro avg ROC curve')
plt.title('ROC curve SVC long-term survivor')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
# plt.savefig('ROC curve SVC long-term survivor.png',dpi=300, facecolor = 'w')
plt.show()

# RFC ROC_AUC

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
import numpy as np

param_grid={'estimator__n_estimators': [30,50,100,200], #100,200,300,400,500
            'estimator__max_features': ['auto'],
            'estimator__max_depth': [4], #Tuning #2,3,4
            'estimator__min_samples_split': [2],#2,3,5
            'estimator__min_samples_leaf':[2],#1,2,5
            'estimator__criterion': ['entropy']}#entropy


rfc = OneVsRestClassifier(RandomForestClassifier(verbose = 2, n_jobs= -1, random_state=42))


rfc_clf = GridSearchCV(
        rfc, param_grid, cv = 10, scoring='roc_auc'
    )

rfc_y_score = rfc_clf.fit(X_train_final, y_train_bi).predict_proba(X_test_final)
y_true, y_pred = y_test_bi, rfc_clf.predict(X_test_final)


In [ ]:
from sklearn.metrics import roc_curve, auc

rfc_fpr = dict()
rfc_tpr = dict()
rfc_roc_auc = dict()
for i in range(n_classes):
    rfc_fpr[i], rfc_tpr[i], _ = roc_curve(y_test_bi[:, i], rfc_y_score[:, i])
    rfc_roc_auc[i] = auc(rfc_fpr[i], rfc_tpr[i])
    
rfc_mean_roc = roc_auc_score(y_true, rfc_clf.predict_proba(X_test_final), multi_class='ovr')
rfc_mean_roc_micro = roc_auc_score(y_true, rfc_clf.predict_proba(X_test_final), multi_class='ovr', average = 'micro')

print("Dictionary of ROC_AUC score per class:")
print(rfc_roc_auc)
print("ROC_AUC score over all three labels [Macro-averaging]:")
print(rfc_mean_roc)
print("ROC_AUC score over all three labels with unbalanced classes taken into account [Micro-averaging]: ")
print(rfc_mean_roc_micro)

In [ ]:
# initializing macro averaging for plotting
import numpy as np

rfc_all_fpr = np.unique(np.concatenate([rfc_fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
rfc_mean_tpr = np.zeros_like(rfc_all_fpr)
for i in range(n_classes):
    rfc_mean_tpr += np.interp(rfc_all_fpr, rfc_fpr[i], rfc_tpr[i])

# Finally average it and compute AUC
rfc_mean_tpr /= n_classes

rfc_fpr["macro"] = rfc_all_fpr
rfc_tpr["macro"] = rfc_mean_tpr
rfc_roc_auc["macro"] = auc(rfc_fpr["macro"], rfc_tpr["macro"])

print(rfc_roc_auc["macro"])

In [ ]:
# initializing micro averaging for plotting

rfc_fpr["micro"], rfc_tpr["micro"], _ = roc_curve(y_test_bi.ravel(), rfc_y_score.ravel())
rfc_roc_auc["micro"] = auc(rfc_fpr["micro"], rfc_tpr["micro"])

print(rfc_roc_auc["micro"])

In [ ]:
#source: https://gist.github.com/aniruddha27/eaf96b6ce2a98eb8cded822d01493a70#file-auc-roc6-py

plt.figure(figsize=(8,8))
plt.plot(rfc_fpr[0], rfc_tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(rfc_fpr[1], rfc_tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(rfc_fpr[2], rfc_tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.plot(rfc_fpr["micro"], rfc_tpr["micro"], linestyle='-',color='red', label='Micro avg ROC curve')
plt.plot(rfc_fpr["macro"], rfc_tpr["macro"], linestyle='-',color='brown', label='Macro avg ROC curve' )
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show()
# plt.savefig('Multiclass ROC',dpi=300);

# GBC ROC_AUC

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
import numpy as np

param_grid={'estimator__n_estimators':[2],
            'estimator__max_depth':[2], 
            'estimator__min_samples_split':[0.1],
            'estimator__min_samples_leaf':[0.1],
            'estimator__max_features':[1,2,3], #89
            'estimator__subsample':[1.0],
            'estimator__learning_rate':[0.05]}

# grid['n_estimators'] = [10, 50, 100, 500]
# grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
# grid['subsample'] = [0.5, 0.7, 1.0]
# grid['max_depth'] = [3, 7, 9]

#optimally tuned parameters
# param_grid={'n_estimators':[100], 'max_depth':[4], 'min_samples_split':[2],
#            'min_samples_leaf':[10],'max_features':[7],'subsample':[1],'learning_rate':[0.005]}


gbc = OneVsRestClassifier(GradientBoostingClassifier(
                                 random_state=42))


gbc_clf = GridSearchCV(gbc, param_grid, cv = 10, scoring='roc_auc')

gbc_y_score = gbc_clf.fit(X_train_final_ha, y_train_bi).decision_function(X_test_final_ha)
y_true, y_pred = y_test_bi, gbc_clf.predict(X_test_final_ha)


gbc_pred_prob = gbc_clf.predict_proba(X_test_final_ha)

In [ ]:
from sklearn.metrics import roc_curve, auc

gbc_fpr = dict()
gbc_tpr = dict()
gbc_roc_auc = dict()
for i in range(n_classes):
    gbc_fpr[i], gbc_tpr[i], _ = roc_curve(y_test_bi[:, i], gbc_y_score[:, i])
    gbc_roc_auc[i] = auc(gbc_fpr[i], gbc_tpr[i])
    
gbc_mean_roc = roc_auc_score(y_true, gbc_clf.predict_proba(X_test_final_ha), multi_class='ovr')
gbc_mean_roc_micro = roc_auc_score(y_true, gbc_clf.predict_proba(X_test_final_ha), multi_class='ovr', average = 'micro')

print("Dictionary of ROC_AUC score per class:")
print(gbc_roc_auc)
print("ROC_AUC score over all three labels [Macro-averaging]:")
print(gbc_mean_roc)
print("ROC_AUC score over all three labels with unbalanced classes taken into account [Micro-averaging]: ")
print(gbc_mean_roc_micro)

In [ ]:
# initializing macro averaging for plotting
import numpy as np

gbc_all_fpr = np.unique(np.concatenate([gbc_fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
gbc_mean_tpr = np.zeros_like(gbc_all_fpr)
for i in range(n_classes):
    gbc_mean_tpr += np.interp(gbc_all_fpr, gbc_fpr[i], gbc_tpr[i])

# Finally average it and compute AUC
gbc_mean_tpr /= n_classes

gbc_fpr["macro"] = gbc_all_fpr
gbc_tpr["macro"] = gbc_mean_tpr
gbc_roc_auc["macro"] = auc(gbc_fpr["macro"], gbc_tpr["macro"])

print(gbc_roc_auc["macro"])

In [ ]:
# initializing micro averaging for plotting

gbc_fpr["micro"], gbc_tpr["micro"], _ = roc_curve(y_test_bi.ravel(), gbc_y_score.ravel())
gbc_roc_auc["micro"] = auc(gbc_fpr["micro"], gbc_tpr["micro"])

print(gbc_roc_auc["micro"])

In [ ]:
#source: https://gist.github.com/aniruddha27/eaf96b6ce2a98eb8cded822d01493a70#file-auc-roc6-py

plt.figure(figsize=(8,8))
plt.plot(gbc_fpr[0], gbc_tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(gbc_fpr[1], gbc_tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(gbc_fpr[2], gbc_tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.plot(gbc_fpr["micro"], gbc_tpr["micro"], linestyle='-',color='red', label='Micro avg ROC curve')
plt.plot(gbc_fpr["macro"], gbc_tpr["macro"], linestyle='-',color='brown', label='Macro avg ROC curve' )
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show()
# plt.savefig('Multiclass ROC',dpi=300);

# Classification accuracy

# Normal k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

knn = KNeighborsClassifier()
knn.fit(X_train_final, y_train_enc)
y_pred = knn.predict(X_test_final)


print(knn.score(X_test_final, y_test_enc)) #mean sub-set accuracy
print(classification_report(y_test_enc, y_pred))
print(confusion_matrix(y_test_enc, y_pred))

# 1. Tuned k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import numpy as np


param_grid = {
    'n_neighbors': np.arange(1,144,1),
}

knn = KNeighborsClassifier()


knn_clf = GridSearchCV(
        knn, param_grid, cv = 10, scoring='accuracy')

knn_clf.fit(X_train_final, y_train_enc)
y_true, y_pred = y_test_enc, knn_clf.predict(X_test_final)
acc = accuracy_score(y_true, y_pred)
knn_roc = roc_auc_score(y_true, y_pred, average = 'micro')

print("Acc:", acc)
print("Accuracy:")
print(knn_clf.score(X_test_final, y_test_enc))
print("Best parameters set found on training set:")
print(knn_clf.best_params_)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

print(knn_roc)

# Normal SVC

In [ ]:
from sklearn.svm import SVC

svc = SVC(random_state=42, class_weight = 'balanced')
svc.fit(X_train_final, y_train_enc)
y_pred = svc.predict(X_test_final)


print(svc.score(X_test_final, y_test_enc)) #mean sub-set accuracy
print(classification_report(y_test_enc, y_pred))
print(confusion_matrix(y_test_enc, y_pred))

# 2. Tuned SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


param_grid = {'kernel': ['linear','rbf','poly','sigmoid'],
              'gamma': [1, 0.1, 0.01,0.001],
              'C': [0.1, 1, 10, 100]}
    

svc = SVC(random_state = 42, class_weight = 'balanced')

svc_clf = GridSearchCV(
        svc, param_grid, cv = 10, scoring='accuracy')

svc_clf.fit(X_train_final, y_train_enc)
y_true, y_pred = y_test_enc, svc_clf.predict(X_test_final)
acc = accuracy_score(y_true, y_pred)
svc_roc = roc_auc_score(y_true, y_pred, average = 'micro')


print("Acc:", acc)
print("Mean accuracy:")
print(svc_clf.score(X_test_final, y_test_enc))
print("Best parameters set found on training set:")
print(svc_clf.best_params_)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))
print(svc_roc)

# Normal RFC

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42, class_weight = 'balanced')
rfc.fit(X_train_final, y_train_enc)


y_pred = rfc.predict(X_test_final)


print(rfc.score(X_test_final, y_test_enc)) #mean sub-set accuracy
print(classification_report(y_test_enc, y_pred))
print(confusion_matrix(y_test_enc, y_pred))

# 3. Tuned RFC

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import numpy as np


param_grid={'n_estimators': [50,100,200,300], #100,200,300,400,500
            'max_features': ['log2'],
            'max_depth': [2,3,4], #Tuning #2,3,4
            'min_samples_split': [2],#2,3,5
            'min_samples_leaf':[5],#1,2,5
            'criterion': ['entropy']}#entropy

# Best parameters set found on training set:
# {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}

rfc = RandomForestClassifier(random_state = 42, verbose=2, n_jobs = -1, class_weight = 'balanced')


rfc_clf = GridSearchCV(
        rfc, param_grid, cv = 10, scoring='accuracy')

rfc_clf.fit(X_train_final, y_train_enc)

y_true, y_pred = y_test_enc, rfc_clf.predict(X_test_final)
acc = accuracy_score(y_true, y_pred)
rfc_roc = roc_auc_score(y_true, y_pred, average = 'micro')

print("Acc:", acc)
print("Mean accuracy:")
print(rfc_clf.score(X_test_final, y_test_enc))
print("Best parameters set found on training set:")
print(rfc_clf.best_params_)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))
print(rfc_roc)

# Normal GBC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
import numpy as np

gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(X_train_final, y_train_enc)


y_pred = gbc.predict(X_test_final)


print(gbc.score(X_test_final, y_test_enc)) #mean sub-set accuracy
print(classification_report(y_test_enc, y_pred))
print(confusion_matrix(y_test_enc, y_pred))

# 5. Tuned GBC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix



param_grid={'n_estimators':[750],
            'max_depth':[3], 
            'min_samples_split':[2,4,6,8,10,20,40,60],
            'min_samples_leaf':[1,3,5,7,9],
            'max_features':['sqrt'],
            'subsample':[1.0],
            'learning_rate':[0.01]}


#optimally tuned parameters
# param_grid={'n_estimators':[100], 'max_depth':[4], 'min_samples_split':[2],
#            'min_samples_leaf':[10],'max_features':[7],'subsample':[1],'learning_rate':[0.005]}


gbc=GradientBoostingClassifier(random_state=42)


gbc_clf = GridSearchCV(
        gbc, param_grid, cv = 10, scoring='accuracy'
)


gbc_clf.fit(X_train_final, y_train_enc)

y_true, y_pred = y_test_enc, gbc_clf.predict(X_test_final)
acc = accuracy_score(y_true, y_pred)
gbc_roc = roc_auc_score(y_true, y_pred, average = 'micro')


print("Acc:", acc)
print("Mean accuracy:")
print(gbc_clf.score(X_test_final, y_test_enc))
print("Best parameters set found on training set:")
print(gbc_clf.best_params_)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))
print(gbc_roc)

In [ ]:
from sklearn import datasets, metrics, model_selection, svm

gbc_roccurve = metrics.plot_roc_curve(gbc_clf, X_test_final, y_test_enc)
plt.show()

ax = plt.gca()
rfc_disp = metrics.plot_roc_curve(rfc_clf, X_test_final, y_test_enc, ax=ax, alpha=0.8)
gbc_roccurve.plot(ax=ax, alpha=0.8)

gbc_roc_macro = roc_auc_score(y_true, y_pred, average = 'macro')
print("macro:",gbc_roc_macro)

